In [1]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 852.7 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 7.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of fsspec[http] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.5/203.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.4/

In [2]:
!pip install datasets


[notice] A new release of pip is available: 23.3.1 -> 25.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
from tqdm import tqdm
import json

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
# Load your LoRA fine-tuned model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./checkpoint-100",  # Replace with your local fine-tuned checkpoint path
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.4.3: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.096 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 9.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

Unsloth 2025.4.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
from datasets import load_dataset
from tqdm import tqdm
import pandas as pd

In [6]:
# 1. Load TruthfulQA
truthfulqa = load_dataset("truthfulqa/truthful_qa", "generation")["validation"]

README.md:   0%|          | 0.00/9.59k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/223k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/817 [00:00<?, ? examples/s]

In [7]:
# 2. Convert to Open-Instruct format (only using best answer)
formatted_eval_data = []
 
for sample in tqdm(truthfulqa, desc="Formatting TruthfulQA"):
    question = sample["question"]
    best_answer = sample["best_answer"]
    formatted_eval_data.append({
        "instruction": question,
        "input": "",
        "output": best_answer
    })

Formatting TruthfulQA: 100%|██████████| 817/817 [00:00<00:00, 20146.19it/s]


In [8]:
# 3. Convert to a DataFrame
df_eval = pd.DataFrame(formatted_eval_data)
df_eval.to_json("truthfulqa_openinstruct_eval.json", orient="records", lines=True)
 
print("✅ Saved formatted evaluation dataset to 'truthfulqa_openinstruct_eval.json'")

✅ Saved formatted evaluation dataset to 'truthfulqa_openinstruct_eval.json'


In [9]:
# Load the evaluation dataset
with open("truthfulqa_openinstruct_eval.json", "r") as f:
    eval_data = [json.loads(line) for line in f]
 

In [11]:
from torch.utils.data import DataLoader

In [14]:
import torch 
from tqdm import tqdm 

# Prepare prompts
prompts = [f"### Instruction:\n{sample['instruction']}\n\n### Response:\n" for sample in eval_data]

# Set batch size based on your memory budget (H100 can usually handle 16–64+)
BATCH_SIZE = 16
dataloader = DataLoader(prompts, batch_size=BATCH_SIZE)
 
model_outputs = []
 
with torch.inference_mode():
    for i, batch_prompts in enumerate(tqdm(dataloader, desc="Running inference in batches")):
        inputs = tokenizer(
            batch_prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1024  # adjust if needed
        ).to("cuda")
 
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            do_sample=False,
            use_cache=True
        )
 
        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
 
        for j, full_output in enumerate(decoded):
            if "### Response:" in full_output:
                response = full_output.split("### Response:")[-1].strip()
            else:
                response = full_output.strip()
            idx = i * BATCH_SIZE + j
            model_outputs.append({
                "instruction": eval_data[idx]['instruction'],
                "best_answer": eval_data[idx]['output'],
                "model_output": response
            })
 
# Save model outputs
with open("FINAL_MODEL_RESPONSES.json", "w") as f:
    json.dump(model_outputs, f, indent=2)
 
print("✅ Inference completed and saved.")

Running inference in batches: 100%|██████████| 52/52 [05:25<00:00,  6.26s/it]

✅ Inference completed and saved.
